# Projet maison n° 4

In [65]:
# imports
import requests
from bs4 import BeautifulSoup
import json

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [4]:
# partie A
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"


def get_prices_from_url(url):
    prices = {}
    
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    if url[-6] == '2':
        class_name = "pure-u-1 pure-u-md-1-3"
    else:
        class_name = "pure-u-1 pure-u-md-1-4"
    for li_tag in soup.find_all("div", {"class":class_name}):
        prices[li_tag.find("h2").text] = {"price":li_tag.find('span', attrs={'class': 'pricing-table-price'}).text.split(" ")[-3],
                                       "storage":li_tag.find('ul', attrs={'class': 'pricing-table-list'}).text.split("\n")[4].split(" ")[0],
                                       "databases":int(li_tag.find('ul', attrs={'class': 'pricing-table-list'}).text.split("\n")[5].split(" ")[0])}
    return prices


get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La ffind_nextion doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [66]:
# partie B-1

URL1 = "https://www.beerwulf.com/fr-fr/p/bieres/st.-feuillien-blond-fut-2l"
URL2 = "https://www.beerwulf.com/fr-fr/p/bieres/newcastle-brown-ale-fut-2l" 


def extract_beer_infos(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    
    beer_infos = {
        'Nom': soup.find('title').text.split("|")[0].split("-")[0][:-1],
        'Style': soup.find_all('dd', class_=['small-6 medium-9 columns'])[0].text.replace("\n", ""),
        'Contenu': int(soup.find('dd', class_=['small-6 medium-9 columns js-beer-volume']).text.split(" ")[0]),
        'Degré d’alcool': float(soup.find_all('dd', class_=['small-6 medium-9 columns'])[1].text.replace(",", ".")[:-1]),
        'Origine': soup.find('dd', class_=['small-6 medium-9 columns js-beer-country']).text.replace("\r\n", "").split(" ")[0],
        'Brasseur': soup.find_all('dd', class_=['small-6 medium-9 columns'])[2].text.replace("\n", "")
    }
    return beer_infos


extract_beer_infos(URL2)

{'Nom': 'Newcastle Brown Ale',
 'Style': 'Bière Brune',
 'Contenu': 2,
 'Degré d’alcool': 4.7,
 'Origine': 'Royaume-Uni',
 'Brasseur': 'Scottish Newcastle'}

In [72]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"


def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    
    # Sequential version (slow):
    base_URL = "https://www.beerwulf.com"
    beers = []
    res = requests.get(URL_BEERLIST_FRANCE)
    json_list_beers = json.loads(res.text)
    
    for item in json_list_beers.get('items'):
        beers.append(extract_beer_infos(base_URL + item["contentReference"]))
    
    return beers

#pour de la parallélisation : multiprocessing, threading (processus léger)
# from multiprocessing import Pool => pour ne pas faire du séquentiel
extract_beer_list_infos(URL_BEERLIST_FRANCE)

[{'Nom': 'La Cristal IPA du Mont Blanc ',
  'Style': 'IPA',
  'Contenu': 33,
  'Degré d’alcool': 4.7,
  'Origine': 'France',
  'Brasseur': 'Mont-Blanc'},
 {'Nom': 'Ninkasi Pale Ale ',
  'Style': 'Bière Ambrée',
  'Contenu': 33,
  'Degré d’alcool': 4.5,
  'Origine': 'France',
  'Brasseur': 'Ninkasi Fabriques'},
 {'Nom': 'Desperados ',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 5.9,
  'Origine': 'France',
  'Brasseur': 'Desperados'},
 {'Nom': 'Pietra ',
  'Style': 'Lager',
  'Contenu': 33,
  'Degré d’alcool': 6.0,
  'Origine': 'France',
  'Brasseur': 'Brasserie Pietra'}]

In [73]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [74]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 3.244s

OK
